# Book Recommendation Project     

Have you ever wondered for a system which suggest a new set of books for your previously read books !?.   
Well [Recommendation Systems](https://en.wikipedia.org/wiki/Recommender_system#:~:text=A%20recommender%20system%2C%20or%20a,would%20give%20to%20an%20item.) are there to help, they are very common to us due to continuous demand and need to get the best book to read.  
A recommendation system is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item.  
They are primarily used in commercial applications. (source - Wikipedia)

In this Project we analyse and preprocess the [Book Crossing Dataset](https://www.kaggle.com/mohitnirgulkar/book-recommendation-data) collected by [Cai-Nicolas Ziegler](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)   
and apply Machine Learning to recommend different books from a book you previously read.   
Whole code below is in [Python](https://www.python.org/) using various libraries. Open source library [Scipy](https://www.scipy.org/) is used for preprocessing and [Scikit-Learn](https://scikit-learn.org/) is used for creating the model.

<p align="center">
    <br clear="right"/>
    <img src="https://d15fwz9jg1iq5f.cloudfront.net/wp-content/uploads/2019/07/31184107/BlogPreview.png" alt="Books" width="800" height="1000" />
</p>

#  Table of Contents


 1. Dependancies and Dataset

 2. Data Cleaning

 3. Data Exploration
 
 4. Popularity Based Recommendation
 
 5. Data Preprocessing
 
 6. Machine Learning Modelling and Output
 
 7. Saving Model and Files

# 1. Dependancies and Dataset

### Importing Dependancies

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import string
import re
import pickle as pkl
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# change defaults
sns.set_context('notebook')
sns.set_style('darkgrid')
sns.set_palette('rainbow')

### Importing Datasets

In [ ]:
#Columns Names
book_cols = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
rating_cols = ['userId','ISBN','bookRating']
user_cols = ['userId','location','age']

books = pd.read_csv('../input/book-recommendation-data/BX-Books.csv',
                    sep=';', error_bad_lines=False, encoding='latin-1')
books.columns = book_cols

ratings = pd.read_csv('../input/book-recommendation-data/BX-Book-Ratings.csv',
                      sep=';', error_bad_lines=False, encoding='latin-1')
ratings.columns = rating_cols

users = pd.read_csv('../input/book-recommendation-data/BX-Users.csv',
                    sep=';', error_bad_lines=False, encoding='latin-1')
users.columns = user_cols

Displaying first two entries of Books data using [DataFrame.head()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)

In [ ]:
books.head(2)

Displaying first two entries of Ratings data

In [ ]:
ratings.head(2)

Displaying first two entries of users Data

In [ ]:
users.head(2)

# 2. Data Cleaning

### Checking Books Data

Using [DataFrame.info()](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.info.html) method to check number of null values and Data types of the data

In [ ]:
books.info()

Column yearOfPublication should be set having dtype as int, hence checking the unique values of yearOfPublication using [Series.unique()](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) method

In [ ]:
books.yearOfPublication.unique()

We can observe from above that some author names are mixed up in year data like 'DK Publishing Inc' and 'Gallimard'  
Checking the rows having 'DK Publishing Inc' as yearOfPublication using [DataFrame.loc[]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) method

In [ ]:
books.loc[books.yearOfPublication == 'DK Publishing Inc',:]

There are Two Books with wrong year entries, Hence we will correct them below

In [ ]:
books.loc[books.ISBN == '078946697X','imageUrlL'] = 'http://images.amazon.com/images/P/078946697X.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '078946697X','imageUrlM'] = 'http://images.amazon.com/images/P/078946697X.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '078946697X','imageUrlS'] = 'http://images.amazon.com/images/P/078946697X.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '078946697X','publisher'] = 'DK Publishing Inc'
books.loc[books.ISBN == '078946697X','yearOfPublication'] = '2000'
books.loc[books.ISBN == '078946697X','bookAuthor'] = 'Michael Teitelbaum'
books.loc[books.ISBN == '078946697X','bookTitle'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'

In [ ]:
books.loc[books.ISBN == '0789466953','imageUrlL'] = 'http://images.amazon.com/images/P/0789466953.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '0789466953','imageUrlM'] = 'http://images.amazon.com/images/P/0789466953.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '0789466953','imageUrlS'] = 'http://images.amazon.com/images/P/0789466953.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '0789466953','publisher'] = 'DK Publishing Inc'
books.loc[books.ISBN == '0789466953','yearOfPublication'] = '2000'
books.loc[books.ISBN == '0789466953','bookAuthor'] = 'James Buckley'
books.loc[books.ISBN == '0789466953','bookTitle'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

Checking the rows having 'Gallimard' as yearOfPublication

In [ ]:
books.loc[books.yearOfPublication == 'Gallimard',:]

In [ ]:
books.loc[books.ISBN == '2070426769','imageUrlL'] = 'http://images.amazon.com/images/P/2070426769.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '2070426769','imageUrlM'] = 'http://images.amazon.com/images/P/2070426769.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '2070426769','imageUrlS'] = 'http://images.amazon.com/images/P/2070426769.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '2070426769','publisher'] = 'Gallimard'
books.loc[books.ISBN == '2070426769','yearOfPublication'] = '2003'
books.loc[books.ISBN == '2070426769','bookAuthor'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'
books.loc[books.ISBN == '2070426769','bookTitle'] = "Peuple du ciel, suivi de 'Les Bergers"

Storing Image URLs in different DataFrame because we don't need them now

In [ ]:
books_data = books[['ISBN', 'bookTitle','imageUrlS', 'imageUrlM', 'imageUrlL']]
books = books[['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher']]

Verifying the changes in publication year 

In [ ]:
books.loc[(books.ISBN == '2070426769') | (books.ISBN == '078946697X') | (books.ISBN == '0789466953'),: ]

Converting year data from object type to numeric using [pd.to_numeric](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html) and if error occurs replacing them with NAN values

In [ ]:
books.yearOfPublication = pd.to_numeric(books.yearOfPublication, errors='coerce')

The value 0 for year is invalid and as this dataset was published in 2004, I have assumed the the years after 2006 to be 
invalid keeping some margin in case dataset was updated thereafer setting invalid years as NaN

In [ ]:
books.loc[(books.yearOfPublication > 2006) | (books.yearOfPublication == 0),'yearOfPublication'] = np.NAN

Using [Series.fillna()](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) to fill the NAN values with mean value of the years and then converting data type to int

In [ ]:
books.yearOfPublication.fillna(round(books.yearOfPublication.mean()),inplace=True)
convert_dict = {'yearOfPublication': np.int64,}
books = books.astype(convert_dict)

Checking if there are NULL values present in publisher column using [Series.isnull()](https://pandas.pydata.org/docs/reference/api/pandas.Series.isnull.html)

In [ ]:
books.loc[books.publisher.isnull(),:]

Checking with rows having bookTitle as Tyrant Moon to see if we can get any clues

In [ ]:
books.loc[(books.bookTitle == 'Tyrant Moon'),:]

No clues for Tyrant Moon , Hence checking with rows having bookTitle as Finders Keepers to see if we can get any clues

In [ ]:
books.loc[(books.bookTitle == 'Finders Keepers'),:]

No clues here either as every book has different publishers , So checking  with rows having author Elaine Corvidae and Linnea Sinclair

In [ ]:
books.loc[(books.bookAuthor == 'Elaine Corvidae') | (books.bookAuthor == 'Linnea Sinclair'),:]

Assinging the Null publisher values as 'Unknown'

In [ ]:
books.loc[(books.ISBN == '193169656X'),'publisher'] = 'Unknown'
books.loc[(books.ISBN == '1931696993'),'publisher'] = 'Unknown'

Checking for Null bookAuthor names 

In [ ]:
books.loc[books.bookAuthor.isnull(),:]

Lets give the Null bookAuthor value as 'Unknown'

In [ ]:
books.loc[(books.ISBN == '9627982032'),'bookAuthor'] = 'Unknown'

### Checking Users Data

Checking for number of null values and data types of data provided inside columns

In [ ]:
users.info()

Age values below 4 and above 90 does not make much sense for our book rating case, hence replacing these by NANs

In [ ]:
users.loc[(users.age > 90) | (users.age < 4), 'age'] = np.nan

Replacing NANs with mean value of users and converting data type to int

In [ ]:
users.age = users.age.fillna(users.age.mean())
users.age = users.age.astype(np.int64)

### Checking Ratings Data

Checking for Null values and data types of Ratings data

In [ ]:
ratings.info()

Looks like it doesn't have any NAN values, So let's have a look at bookRating values

In [ ]:
ratings.bookRating.unique()

bookRatings are always under 0-10, also Ratings dataset will have n_users $\times$ n_books entries if every user rated every item,

In [ ]:

n_users = users.shape[0]
n_books = books.shape[0]
print(n_users * n_books)

From above we can say that the dataset is very sparse,  
Ratings dataset should have books only which exist in our books dataset, unless new books are added to books dataset

In [ ]:
ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]

Also Ratings dataset should have ratings from users which exist in users dataset, unless new users are added to users dataset

In [ ]:
ratings_new = ratings_new[ratings_new.userId.isin(users.userId)]
#Note: All users who rated are already inside users dataset

Dataset [Sparsity](https://campus.datacamp.com/courses/recommendation-engines-in-pyspark/recommending-movies?ex=3) Calculation

In [ ]:
sparsity = 1 - (len(ratings_new)/(n_users*n_books))
print("No. of users = " + str(n_users) + ", No. of Books = " + str(n_books) 
      + "\nThe Following Dataset has " + str(sparsity*100) + " % Sparsity")

Making two new ratings dataframes where ratings which are other than 0 are present and vice a versa

In [ ]:
rating_explicit = ratings_new.loc[ratings_new.bookRating != 0, :]
rating_implicit = ratings_new.loc[ratings_new.bookRating == 0, :]
print("Explicit Ratings data shape = " + str(rating_explicit.shape) + "\nImplicit Ratings data shape = " + str(rating_implicit.shape))

### Visualising Explicit Rating Counts

Using [Seaborn](https://towardsdatascience.com/seaborn-python-8563c3d0ad41) Library for ploting a [countplot](https://www.geeksforgeeks.org/countplot-using-seaborn-in-python/)

In [ ]:
plt.figure(figsize=(9, 5))
plt.title('Explicit Rating Counts')
sns.countplot(x = 'bookRating', data = rating_explicit);

### Cleaning Title text

Function for cleaning title text using [Regex](https://en.wikipedia.org/wiki/Regular_expression) Library

In [ ]:
def clean_text(text):
    #removal of url
    text = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ',text) 
    
    #removal of html tags
    text = re.sub(r'<.*?>',' ',text) 
    
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',text)
    
    
    #remove newline char
    text = re.sub('\n', '', text)
    
    return text

Applying clean_text() function on bookTitle column

In [ ]:
books.bookTitle = books.bookTitle.apply(lambda x:clean_text(x))

Merging Books and Ratings data using [pd.merge()](https://www.geeksforgeeks.org/python-pandas-merging-joining-and-concatenating/)

In [ ]:
merged_data = pd.merge(books, ratings_new, on='ISBN')
merged_data =  merged_data.sort_values('ISBN', ascending=True)
merged_data.head()

# 3. Data Exploration

### Visualising top 30 most read books

Using [Series.value_counts()](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) and [sns.barplot](https://seaborn.pydata.org/generated/seaborn.barplot.html) for creating data and visualisation respectively

In [ ]:
most_read = merged_data.bookTitle.value_counts().reset_index()
most_read.columns = ['bookTitle','count']

plt.figure(figsize = (10,10))
plt.title("Most Read Books")
sns.barplot(x = 'count', y = 'bookTitle', data = most_read.head(30));

### Top rated books(average rating according to number of users)

We may observe many books with average rating equal to 10 and 0 as many of the books are rated only once, hence this can't show us a good visualisation

In [ ]:
top_rated = merged_data[['bookTitle','bookRating']]
top_rated = top_rated.groupby('bookTitle', as_index=False)['bookRating'].mean()
top_rated = top_rated.sort_values('bookRating',ascending=False).reset_index()
top_rated = top_rated[['bookTitle','bookRating']]

top_rated.head()

In [ ]:
top_rated.tail()

### Visualising top 30 most read books with there average ratings

Using [DataFrame.groupby()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) and [DataFrame.sort_values()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) for ploting a barplot

In [ ]:
avg_data = merged_data.groupby('bookTitle', as_index=False)['bookRating'].mean()
temp = merged_data.bookTitle.value_counts().reset_index()
temp.columns = ['bookTitle','count']
most_rated_by_reads = pd.merge(avg_data,temp,on='bookTitle')

most_rated_by_reads = most_rated_by_reads.sort_values('count',ascending=False)

plt.figure(figsize=(12,10))
plt.title("Average Ratings of Most Read books")
sns.barplot(x = 'bookRating', y = 'bookTitle', data = most_rated_by_reads.head(30));

### Visualising Count of Books with a specific length

Creating title_length data for every title in bookTitle column and ploting a histplot

In [ ]:
title_length = books.bookTitle.str.split().map(lambda x: len(x))
plt.figure(figsize=(8,6))
plt.title('Number of books with a specific title length')
sns.histplot(title_length, bins=25, color = 'lightgreen', linewidth = 2, edgecolor = 'black');

### Visualising top 30 years with most book being published

We count the number of times a book is published in a particular year and plot a barplot

In [ ]:
yearOP = books['yearOfPublication'].value_counts().reset_index()

yearOP.columns = ['value', 'count']

yearOP['year'] = yearOP['value'].astype(str) + ' year'

yearOP = yearOP.sort_values('count',ascending=False)

plt.figure(figsize=(12,10))
plt.title('Top 30 years of publishing')
sns.barplot(data = yearOP.head(30), x='count', y="year");

### Visualising top 30 authors with most books

We count the number of times a book written by an author and plot a barplot for the top 30 authors

In [ ]:
top_author = books['bookAuthor'].value_counts().reset_index()
top_author.columns = ['Author', 'count']
top_author['Author'] = top_author['Author']
top_author = top_author.sort_values('count',ascending=False)

plt.figure(figsize=(12,10))
plt.title('Top 30 Authors according to most books')
sns.barplot(data = top_author.head(30), x = 'count', y = 'Author');

### Visualising top 30 publishers with most books

We count the number of books published by a publisher and plot top 30 of them

In [ ]:
top_publisher = books['publisher'].value_counts().reset_index()
top_publisher.columns = ['Publisher', 'count']
top_publisher['Publisher'] = top_publisher['Publisher']
top_publisher = top_publisher.sort_values('count',ascending=False)

plt.figure(figsize=(12,10))
plt.title('Top 30 Publishers according to most books')
sns.barplot(data = top_publisher.head(30), x = 'count', y = 'Publisher');

### Visualising the age distribution of the users

In [ ]:
user_age = users.age
plt.figure(figsize=(10,6), tight_layout=True)
plt.title('Number of users according to user age')
sns.histplot(user_age, bins=25, linewidth = 2, color = 'cyan', edgecolor = 'black');

### Visualising most frequent words in Author, Title and Publisher string

Creating string variables for bookAuthor, bookTitle and publisher, also creating a function to plot [WordCloud()](https://www.geeksforgeeks.org/generating-word-cloud-python/) 

In [ ]:
author_string = " ".join(books['bookAuthor'].astype(str))
title_string = " ".join(books['bookTitle'].astype(str))
publisher_string = " ".join(books['publisher'].astype(str))
stop_words = set(STOPWORDS)

def wordcloud(string,title ="unknown"):
    wc = WordCloud(width=800,height=500,mask=None,random_state=21,
                   stopwords=stop_words).generate(string)
    fig=plt.figure(figsize=(16,8))
    plt.title(title)
    plt.axis('off')
    plt.imshow(wc)

Displaying most frequent words in author names

In [ ]:
wordcloud(author_string,'Author Name words')

Displaying most frequent words in title text

In [ ]:
wordcloud(title_string,'Title text words')

Displaying most frequent words in publisher names

In [ ]:
wordcloud(publisher_string,'Publisher Name words')

# 4. Popularity Based Recommendation

Below we show top 10 recommendations based on popularity using sum of bookRating values to choose which is popular.    
It is evident that books authored by [J.K. Rowling](https://www.jkrowling.com/) are one of the most popular

In [ ]:
ratings_count = pd.DataFrame(rating_explicit.groupby(['ISBN'])['bookRating'].sum())
top_10 = ratings_count.sort_values('bookRating', ascending = False).head(10)
print("Following books are recommended")
top_10.merge(books, left_index = True, right_on = 'ISBN').reset_index()

### If you are having small dataset it's worth trying code below using cosine similarity matrix for content based recommendation

In [ ]:
# #Content Based Recommendation 
# # Be sure to import the libraries needed
# content_data = books[['bookTitle','bookAuthor','publisher']]
# content_data = content_data.astype(str)
# content_data['content'] = content_data['bookTitle'] + ' ' + content_data['bookAuthor'] + ' ' + content_data['publisher']
# content_data = content_data.reset_index()
# indices = pd.Series(content_data.index, index=content_data['bookTitle'])

In [ ]:
# count = CountVectorizer(stop_words='english')

# count_matrix = count.fit_transform(content_data['content'])

# cosine_sim_content = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# def get_recommendations(title, cosine_sim=cosine_sim_content):
#     idx = indices[title]

#     # Get the pairwsie similarity scores of all books with that book
#     sim_scores = list(enumerate(cosine_sim_content[idx]))

#     # Sort the books based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar books
#     sim_scores = sim_scores[1:11]

#     # Get the book indices
#     book_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar books
#     return list(content_data['original_title'].iloc[book_indices])

# def book_shows(book):
#     for book in book:
#         print(book)

# 5. Data Preprocessing

Users with less than 100 ratings, and books with less than 100 ratings are excluded

In [ ]:
counts1 = ratings_new['userId'].value_counts()
ratings_data = ratings_new[ratings_new['userId'].isin(counts1[counts1 >= 100].index)]
counts = ratings_data['bookRating'].value_counts()
ratings_data = ratings_data[ratings_data['bookRating'].isin(counts[counts >= 100].index)]

With the new ratings data we merge the Books data using pd.merge() and drop irrelavant columns

In [ ]:
merged_new = pd.merge(ratings_data,books,on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor']
merged_new = merged_new.drop(columns, axis=1)

Creating a DataFrame number_rating with count of number of times a book is being rated   
[Using DataFrame.rename()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) to rename columns

In [ ]:
number_rating = merged_new.groupby('bookTitle')['bookRating'].count().reset_index()
number_rating.rename(columns={'bookRating':'number of rating'},inplace=True)
number_rating.head()

Creating final_ratings DataFrame for every bookTitle with bookRatings and number of ratings

In [ ]:
final_ratings = merged_new.merge(number_rating,on='bookTitle') 
final_ratings.head()

We consider those book who have been rated more than 30 times

In [ ]:
final_ratings = final_ratings[final_ratings['number of rating']>=30] 

Droping duplicate records from final_ratings using [DataFrame.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) and checking the change of DataFrame size

In [ ]:
print("Shape Before :" + str(final_ratings.shape))
final_ratings.drop_duplicates(['userId','bookTitle'],inplace=True)
print("Shape After  :" + str(final_ratings.shape))

Creating  a pivot table for final_ratings dataframe with columns as userId and index as bookTitle with values as bookRatings with the help of [DataFrame.pivot_table()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

In [ ]:
book_pivot = final_ratings.pivot_table(columns='userId',index='bookTitle',values='bookRating')
book_pivot.shape

Filling the NAN values with zeros

In [ ]:
book_pivot.fillna(0,inplace=True)
book_pivot

# 6. Machine Learning Modelling and Output

Using [scipy](https://www.scipy.org/) library's [compressed sparse row matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) to create sparse book matrix with book_pivot as input   
We then use this sparse matrix to provide an input to the scikit-learn's [NearestNeighbors model](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) with [brute algorithm](https://www.kdnuggets.com/2020/10/exploring-brute-force-nearest-neighbors-algorithm.html) and [metric cosine](https://www.machinelearningplus.com/nlp/cosine-similarity/).

In [ ]:
book_sparse=csr_matrix(book_pivot)
model=NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(book_sparse)

Creating a function which will recommend 10 new books for the book you read according to the suggestions provided the model

In [ ]:
def recommend(book_name):
    book_id = np.where(book_pivot.index==book_name)[0][0]
    distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors = 11)
    
    
    books=[]
    for i in range(len(suggestions)):
        if i==0:
            print("The suggestions for ",book_name,"are : ")
        if not i:
            books = book_pivot.index[suggestions[i]]
    for i in range(1,len(books)):
         print(str(i) + ": " + books[i] )       
                
            
recommend('The Sum of All Fears (Jack Ryan Novels)')

# 7. Saving Model and Files


In [ ]:
books_name_Murl = books_data[['bookTitle', 'imageUrlM']]

In [ ]:
books_name_Murl.to_csv('Book_names_with_urlM.csv')

final_ratings.to_csv('Final_Ratings.csv')

with open('model.pkl', 'wb') as file:
      
    # A new file will be created
    pkl.dump(model, file)
    

# Upvote this notebook if it help you in any way